<a href="https://colab.research.google.com/github/krixik-ai/krixik-docs/blob/main/docs/system/search_methods/semantic_search_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import json
import importlib
from pathlib import Path

# preparación de demo - incuye instanciación de secretos, instalación de requerimientos, y definición de rutas
if os.getenv("COLAB_RELEASE_TAG"):
    # si estás usando este notebook en Google Colab, ingresa tus secretos acá
    MY_API_KEY = "TU_API_KEY_VA_AQUI"
    MY_API_URL = "TU_API_URL_VA_AQUI"

    # si estás usando este notebook en Google Colab, instala requerimientos y descarga los subdirectorios requeridos
    # instala el cliente Python de Krixik
    !pip install krixik

    # instala github-clone, que permite clonación fácil de los subdirectorios del repositorio de documentación XXX
    !pip install github-clone

    # clona los conjuntos de datos
    if not Path("data").is_dir():
        !ghclone XXXX #(in english it's https://github.com/krixik-ai/krixik-docs/tree/main/data)
    else:
        print("ya se clonaron los conjuntos de datos de documentación!")

    # define la variable 'data_dir' para tus rutas
    data_dir = "./data/"

    # crea directorio de salidas
    from pathlib import Path

    Path(data_dir + "/salidas").mkdir(parents=True, exist_ok=True)

    # descarga utilidades
    if not Path("utilities").is_dir():
        !ghclone XXXX # (in english it's https://github.com/krixik-ai/krixik-docs/tree/main/utilities)
    else:
        print("ya has clonado las utilidades de documentación!")
else:
    # si estás usando una descarga local de la documentación, define las rutas relativas a la estructura local de la documentación
    # importa utilidades
    sys.path.append("../../../")

    # define la variable 'data_dir' para tus rutas
    data_dir = "../../../data/"

    # si estás usando este notebook localmente desde el repositorio de documentación Krixik, carga tus secretos de un archivo .env ubicado en la base del repositorio de documentación
    from dotenv import load_dotenv

    load_dotenv("../../../.env")

    MY_API_KEY = os.getenv("MY_API_KEY")
    MY_API_URL = os.getenv("MY_API_URL")


# carga 'reset'
reset = importlib.import_module("utilities.reset")
reset_pipeline = reset.reset_pipeline


# importa Krixik e inicializa sesión con tus secretos personales
from krixik import krixik

krixik.init(api_key=MY_API_KEY, api_url=MY_API_URL)

SUCCESS: You are now authenticated.


## El Método `semantic_search` (Búsqueda Semántica)

El método `semantic_search` de Krixik habilita búsqueda semántica sobre documentos procesados a través de ciertos *pipelines*. Mucho se ha escrito sobre la búsqueda semántica, pero en breve, en vez de buscar palabras clave en un documento, este método busca texto que es similar en _significado_ al *string* que se ha enviado. Esto es diferente a lo que ofrece el [`keyword_search`](metodo_keyword_search_busqueda_por_palabras_clave.md).

Dado que el método `semantic_search` hace [`encaje léxico`](../../modulos/modulos_ia/modulo_text-embedder_encaje_lexico.md) con el *string* enviado (*the query*) y también hace la búsqueda, solo se puede usar con *pipelines* que contienen un módulo [`text embedder` (encaje léxico)](../../modulos/modulos_ia/modulo_text-embedder_encaje_lexico.md) y un módulo [`vector-db` (base de datos vectorial)](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md) en secuencia inmediata.

Esta introducción al método `semantic_search` está dividida en las siguientes secciones:

- [Argumentos del Método `semantic_search`](#argumentos-del-metodo-semantic_search)
- [Ejemplo de Montaje de Pipeline y Procesamiento de Archivo](#ejemplo-de-montaje-de-pipeline-y-procesamiento-de-archivo)
- [Ejemplos de Búsqueda Semántica](#ejemplos-de-busqueda-semantica)
- [Límite de Tamaño de Salidas](#limite-de-tamano-de-salidas)

### Argumentos del Metodo `semantic_search`

El método `semantic_search` toma un argumento requerido y al menos uno de varios argumentos opcionales. El argumento requerido es:

- `query` (str) - Un *string* cuyo significado será el objeto de búsqueda en el documento señalado. Las coincidencias más cercanas (es decir, los fragmentos que más se acercan al query en su significado) serán devueltas.

Los argumentos opcionales son los mismos argumentos que el método [`list`](../sistema_de_archivos/metodo_list_lista.md) recibe—tanto los de metadata como las marcas de tiempo—así que [detállalos aquí](../sistema_de_archivos/metodo_list_lista.md#argumentos-del-metodo-list-lista) si es necesario. Al igual que con el método [`list`](../sistema_de_archivos/metodo_list_lista.md), puedes hacer búsqueda semántica sobre varios archivos a la vez porque todos los argumentos de metadata se envían al método `semantic_search` en formato de lista. Todos los elementos de los argumentos opcionales son iguales que para el método [`list`](../sistema_de_archivos/metodo_list_lista.md), incluyendo el operador comodín * y la raíz global.

Si no está presente ninguno de estos argumentos opcionales, el método `semantic_search` no funcionará porque no hay dónde buscar.

Al igual que el método [`list`](../sistema_de_archivos/metodo_list_lista.md), el método `semantic_search` acepta los argumentos opcionales `max_files` y `sort_order`, aunque su función cambia un poco:

- `max_files` especifica en hasta cuántos archivos se debe buscar. Su valor predeterminado no existe; no habría un máximo.

- `sort_order` acá toma tres valores posibles: 'ascending', 'descending', y 'global'. Los primeros dos ordenan los resultados por el archivo en el que están (los archivos se ordenan por su marca de tiempo de creación), y 'global' combina todos los archivos y devuelve los mejores resultados de entre la totalidad de archivos. Su valor predeterminado es 'descending'.

El método `semantic_search` recibe un argumento opcional que es único a este método:

- `k` (int) - Especifica hasta cuántos resultados se deben devolver por archivo consultado. Su valor predeterminado es 5.

### Ejemplo de Montaje de Pipeline y Procesamiento de Archivo

Para los ejemplos de este documento usarás un *pipeline* que consiste de tres módulos: un módulo [`parser`](../../modulos/modulos_de_funciones_de_apoyo/modulo_parser_fragmentacion.md), un módulo [`text-embedder`](../../modulos/modulos_ia/modulo_text-embedder_encaje_lexico.md) y un módulo [`vector-db`](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md). Este es el [*pipeline* básico de búsqueda semántica](../../ejemplos/ejemplos_pipelines_de_busqueda/multi_busqueda_semantica_basica.md). Usa el método [`create_pipeline`](../creacion_de_pipelines/creacion_de_pipelines.md) para crearlo:

In [2]:
# crea el pipeline básico de búsqueda semántica
pipeline = krixik.create_pipeline(name="metodo_semantic_search_1", module_chain=["parser", "text-embedder", "vector-db"])

Una vez creado el *pipeline*, puedes [`procesar`](../parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md) algunos archivos de texto a través de él para tener sobre qué buscar:

In [3]:
# agrega cuatro archivos al pipeline que acabas de crear
salida_1 = pipeline.process(
    local_file_path=data_dir + "input/frankenstein_muy_corto.txt",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False,  # no mostrar actualizaciones de proceso al ejecutar el código
    symbolic_directory_path="/novelas/gotica",
    file_name="Frankenstein.txt",
)

salida_2 = pipeline.process(
    local_file_path=data_dir + "input/orgullo_y_prejuicio_muy_corto.txt",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False,  # no mostrar actualizaciones de proceso al ejecutar el código
    symbolic_directory_path="/novelas/romance",
    file_name="Pride and Prejudice.txt",
)

salida_3 = pipeline.process(
    local_file_path=data_dir + "input/moby_dick_muy_corto.txt",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False,  # no mostrar actualizaciones de proceso al ejecutar el código
    symbolic_directory_path="/novelas/aventura",
    file_name="Moby Dick.txt",
)

salida_4 = pipeline.process(
    local_file_path=data_dir + "input/mujercitas_muy_corto.txt",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False,  # no mostrar actualizaciones de proceso al ejecutar el código
    symbolic_directory_path="/novelas/bildungsroman",
    file_name="Little Women.txt",
)

Examina la salida de uno de estos:

In [4]:
# nítidamente reproduce la salida de este proceso
print(json.dumps(salida_2, indent=2))

{
  "status_code": 200,
  "pipeline": "semantic_search_method_1_parser_text-embedder_vector-db",
  "request_id": "4197e750-0560-43b9-b7e3-0ea5c8f15151",
  "file_id": "a94765c2-0250-4b3d-98af-20fc167640e8",
  "message": "SUCCESS - output fetched for file_id a94765c2-0250-4b3d-98af-20fc167640e8.Output saved to location(s) listed in process_output_files.",
  "warnings": [],
  "process_output": null,
  "process_output_files": [
    "../../../data/output/a94765c2-0250-4b3d-98af-20fc167640e8.faiss"
  ]
}


El valor de `process_output` es `null` porque el objeto devuelto es una base de datos, así que no se puede reproducir aquí. Puedes encontrar ese archivo de base de datos en la ubicación local indicada en `process_output_files`.

### Ejemplos de Busqueda Semantica

Ahora que has procesado archivos por el *pipeline* puedes usar el método `semantic_search` sobre él.

Con el siguiente código puedes buscar semánticamente sobre uno de los archivos:

In [5]:
# haz semantic_search sobre un archivo
semantic_output = pipeline.semantic_search(query="It was cold night.", file_names=["Little Women.txt"])

# nítidamente reproduce la salida de este proceso
print(json.dumps(semantic_output, indent=2))

{
  "status_code": 200,
  "request_id": "c1b9116f-0eaa-489d-a8f4-86ca7238e744",
  "message": "Successfully queried 1 user file.",
  "warnings": [],
  "items": [
    {
      "file_id": "853f498f-4b1c-439b-bbd4-ccc47c44d254",
      "file_metadata": {
        "file_name": "little women.txt",
        "symbolic_directory_path": "/novels/bildungsroman",
        "file_tags": [],
        "num_vectors": 43,
        "created_at": "2024-06-05 16:19:43",
        "last_updated": "2024-06-05 16:19:43"
      },
      "search_results": [
        {
          "snippet": "The four young faces on which the firelight shone brightened at the\ncheerful words, but darkened again as Jo said sadly,--\n\n\"We haven't got father, and shall not have him for a long time.\"",
          "line_numbers": [
            19,
            20,
            21,
            22,
            23
          ],
          "distance": 0.351
        },
        {
          "snippet": "Nobody spoke for a minute; then Meg said in an altere

Además de devolver los fragmentos cuyo significado más se acercan al string enviado (*the query*), el método `semantic_search` también devuelve la distancia calculada entre vectores (que puede entenderse como la "distancia entre significados") de resultado y *query*. Mientras más corta es esta distancia, más se acerca el significado de este fragmento al del *query*. El método `semantic_search` devuelve los fragmentos con la distancia de vector más corta al *query*, ordenado de manera ascendiente dentro de cada archivo.

Cuando el argumento `sort_order` tiene el valor 'global', los resultados de todos los archivos se combinan y el método devuelve los fragmentos con la distancia más corta al *query*, ordenados de manera ascendiente, sin importar en qué archivo estén. Inténtalo haciéndo una búsqueda semántica sobre varios archivos con el [operador comodín](../sistema_de_archivos/metodo_list_lista.md#argumentos-con-el-operador-comodin):

In [6]:
# haz búsqueda semántica sobre varios archivos
semantic_output_2 = pipeline.semantic_search(query="It was cold night.", symbolic_directory_paths=["/novelas*"], sort_order="global", k=4)

# nicely print the output of this search
print(json.dumps(semantic_output_2, indent=2))

{
  "status_code": 200,
  "request_id": "ba1b7b85-8e36-49e5-8734-68c80d19e433",
  "message": "Successfully queried 4 user files.",
  "warnings": [],
  "items": [
    {
      "snippet": "I am already far north of London, and as I walk in the streets of\nPetersburgh, I feel a cold northern breeze play upon my cheeks, which\nbraces my nerves and fills me with delight.",
      "distance": 0.33,
      "line_numbers": [
        14,
        15,
        16,
        17
      ],
      "file_metadata": {
        "file_id": "f4720361-f94f-4f48-a4bf-0177dd91ba18",
        "file_name": "frankenstein.txt",
        "symbolic_directory_path": "/novels/gothic",
        "file_tags": [],
        "num_lines": 0,
        "created_at": "2024-06-05 16:17:58",
        "last_updated": "2024-06-05 16:17:58"
      }
    },
    {
      "snippet": "This breeze, which has travelled from the regions towards\nwhich I am advancing, gives me a foretaste of those icy climes.",
      "distance": 0.336,
      "line_numbers

Puedes ver que los resultados de todos los archivos se han combinado, y que el fragmento en el primer lugar tiene la distancia más corta entre *query* y fragmento de todos los archivos incluidos.

### Limite de Tamano de Salidas

El límite actual sobre salidas generadas por el método `semantic_search` es 5MB.

In [7]:
# elimina todos los datos procesados pertenecientes a este pipeline
reset_pipeline(pipeline)